In [1]:
import numpy as np 
import pandas as pd
import os
import math
import random
import cv2

import matplotlib.pyplot as plt
import quandl
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data

import ColorLog as debug

from load_data import labelFpsDataLoader, labelTestDataLoader

In [2]:
DEVICE = "cuda"
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
             'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

NUM_PROV = len(provinces)
NUM_ALPB = len(alphabets)
NUM_ADS = len(ads)



In [10]:
# data loading 
# image size 720x1160x3 


image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")


def list_images(basePath, contains=None):
    # return the set of files that are valid
    return list_files(basePath, validExts=image_types, contains=contains)


def list_files(basePath, validExts=None, contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if validExts is None or ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename)
                yield imagePath
                
class labelFpsDataLoader(Data.Dataset):
    def __init__(self, img_dir, imgSize, is_transform=None):
        self.img_dir = img_dir
        self.img_paths = []
        for i in range(len(img_dir)):
            self.img_paths += [el for el in list_images(img_dir[i])]
        # self.img_paths = os.listdir(img_dir)
        # print self.img_paths
        self.img_size = imgSize
        self.is_transform = is_transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_name = self.img_paths[index]
        img = cv2.imread(img_name)
        # img = img.astype('float32')
        lbl = img_name.split('/')[-1].rsplit('.', 1)[0].split('-')[-3]

        iname = img_name.rsplit('/', 1)[-1].rsplit('.', 1)[0].split('-')
        # fps = [[int(eel) for eel in el.split('&')] for el in iname[3].split('_')]
        # leftUp, rightDown = [min([fps[el][0] for el in range(4)]), min([fps[el][1] for el in range(4)])], [
        #     max([fps[el][0] for el in range(4)]), max([fps[el][1] for el in range(4)])]
        
#         print(debug.DEBUG,iname)
        
        [leftUp, rightDown] = [[int(eel) for eel in el.split('&')] for el in iname[2].split('_')]
        ori_w, ori_h = [float(int(el)) for el in [img.shape[1], img.shape[0]]]
        new_labels = [(leftUp[0] + rightDown[0]) / (2 * ori_w), (leftUp[1] + rightDown[1]) / (2 * ori_h),
                      (rightDown[0] - leftUp[0]) / ori_w, (rightDown[1] - leftUp[1]) / ori_h]
        croppedImage = img[leftUp[1]:rightDown[1],leftUp[0]:rightDown[0]]
        resizedImage = cv2.resize(croppedImage, self.img_size)
#         cv2.imshow('plate',resizedImage)
#         cv2.waitKey(0)
        print(resizedImage.shape)
        resizedImage = np.transpose(resizedImage, (2,0,1))
        resizedImage = resizedImage.astype('float32')
        resizedImage /= 255.0
        
#         cv2.imshow('plate',np.transpose(resizedImage, (1,2,0)))
#         cv2.waitKey(0)
        
        return resizedImage, new_labels, lbl, img_name, iname
    
class labelLoader(Data.Dataset):
    def __init__(self, img_dir, imgSize, is_transform=None):
        self.img_dir = img_dir
        self.img_paths = []
        for i in range(len(img_dir)):
            self.img_paths += [el for el in list_images(img_dir[i])]
        # self.img_paths = os.listdir(img_dir)
        # print self.img_paths
        self.img_size = imgSize
        self.is_transform = is_transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_name = self.img_paths[index]
#         img = cv2.imread(img_name)
#         # img = img.astype('float32')
#         resizedImage = cv2.resize(img, self.img_size)
#         resizedImage = np.transpose(resizedImage, (2,0,1))
#         resizedImage = resizedImage.astype('float32')
#         resizedImage /= 255.0
        lbl = img_name.split('/')[-1].rsplit('.', 1)[0].split('-')[-3]

        iname = img_name.rsplit('/', 1)[-1].rsplit('.', 1)[0].split('-')
        # fps = [[int(eel) for eel in el.split('&')] for el in iname[3].split('_')]
        # leftUp, rightDown = [min([fps[el][0] for el in range(4)]), min([fps[el][1] for el in range(4)])], [
        #     max([fps[el][0] for el in range(4)]), max([fps[el][1] for el in range(4)])]
        
#         print(debug.DEBUG,iname)
        
        [leftUp, rightDown] = [[int(eel) for eel in el.split('&')] for el in iname[2].split('_')]
#         ori_w, ori_h = [float(int(el)) for el in [img.shape[1], img.shape[0]]]
#         new_labels = [(leftUp[0] + rightDown[0]) / (2 * ori_w), (leftUp[1] + rightDown[1]) / (2 * ori_h),
#                       (rightDown[0] - leftUp[0]) / ori_w, (rightDown[1] - leftUp[1]) / ori_h]

        return lbl, img_name, iname

In [14]:
path = "CCPD2019/ccpd_base"
imgsize = (480,480)
platesize = (252,96)
print(debug.INFO+"start loading data...")
dst = labelFpsDataLoader([path],platesize)
# dst = labelLoader([path],imgsize)
print(debug.INFO+"finish loading")
# sizes = []
# for _,_,iname in dst:
#     [leftUp, rightDown] = [[int(eel) for eel in el.split('&')] 
#                        for el in iname[2].split('_')]
#     sizes.append([leftUp[0]-rightDown[0],leftUp[1]-rightDown[1]])

# mean, std = np.mean(sizes, axis=0),np.std(sizes, axis=0)
# print(mean,std)
#     [-252.62252245  -94.97832957] [61.46956803 27.42365916]
for img,_,_,_,iname in dst:
    print(img.shape)
    img =np.transpose(img, (1,2,0))
    
    print(img.shape)
    cv2.imshow('plate',img)
    cv2.waitKey(0)
    break


[INFO] -- start loading data...
[INFO] -- finish loading
(96, 252, 3)
(3, 96, 252)
(96, 252, 3)


In [16]:
class DigitRecog(nn.Module): 
    def __init__(self, ):

        super(DigitRecog, self).__init__()
        self.name = "DigitRecog"
        
        self.stack1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
            nn.Dropout(0.2)
        )
        self.stack2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
            nn.Dropout(0.2)
        )
        self.stack3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
            nn.Dropout(0.2)
        )
        self.stack4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
            nn.Dropout(0.2)
        )
        self.stack5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
            nn.Dropout(0.2)
        )
        self.activations = nn.Sequential(
            self.stack1,
            self.stack2,
            self.stack3,
            self.stack4,
            self.stack5
        )
        
        
        self.classifier1 = nn.Sequential(
            nn.Linear(23232, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, NUM_PROV)
        )
        self.classifier2 = nn.Sequential(
            nn.Linear(23232, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, NUM_ALPB)
        )
        
        self.classifier3 = nn.Sequential(
            nn.Linear(23232, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, NUM_ADS)
        )
        self.classifier4 = nn.Sequential(
            nn.Linear(23232, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, NUM_ADS)
        )
        self.classifier5 = nn.Sequential(
            nn.Linear(23232, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, NUM_ADS)
        )
        self.classifier6 = nn.Sequential(
            nn.Linear(23232, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, NUM_ADS)
        )
        self.classifier7 = nn.Sequential(
            nn.Linear(23232, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, NUM_ADS)
        )
    def forward(self, x):
        x = self.activations(x)
        x = x.view(x.size(0),-1)
        prov = self.classifier1(x)
        alpb = self.classifier2(x)
        ads1 = self.classifier3(x)
        ads2 = self.classifier4(x)
        ads3 = self.classifier5(x)
        ads4 = self.classifier6(x)
        ads5 = self.classifier7(x)
        return prov,alpb,ads1,ads2,ads3,ads4,ads5   


In [ ]:


def train_model(model, trainloader, criterion, optimizer, num_epochs=25):
    # since = time.time()
    for epoch in range(epoch_start, num_epochs):
        lossAver = []
        model.train(True)
        lrScheduler.step()
        start = time()

        for i, (XI, Y, labels, ims) in enumerate(trainloader):
            if not len(XI) == batchSize:
                continue

            YI = [[int(ee) for ee in el.split('_')[:7]] for el in labels]
            Y = np.array([el.numpy() for el in Y]).T
            if use_gpu:
                x = Variable(XI.cuda(0))
                y = Variable(torch.FloatTensor(Y).cuda(0), requires_grad=False)
            else:
                x = Variable(XI)
                y = Variable(torch.FloatTensor(Y), requires_grad=False)
            # Forward pass: Compute predicted y by passing x to the model

            try:
                fps_pred, y_pred = model(x)
            except:
                continue
                
            # Compute and print loss
            loss = 0.0
            loss += 0.8 * nn.L1Loss().cuda()(fps_pred[:][:2], y[:][:2])
            loss += 0.2 * nn.L1Loss().cuda()(fps_pred[:][2:], y[:][2:])
            for j in range(7):
                l = Variable(torch.LongTensor([el[j] for el in YI]).cuda(0))
                loss += criterion(y_pred[j], l)

            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            try:
                lossAver.append(loss.data[0])
            except:
                pass

            if i % 50 == 1:
                with open(args['writeFile'], 'a') as outF:
                    outF.write('train %s images, use %s seconds, loss %s\n' % (i*batchSize, time(    ) - start, sum(lossAver) / len(lossAver) if len(lossAver)>0 else 'NoLoss'))
                torch.save(model.state_dict(), storeName)
        print ('%s %s %s\n' % (epoch, sum(lossAver) / len(lossAver), time()-start))
        model.eval()
        count, correct, error, precision, avgTime = eval(model, testDirs)
        with open(args['writeFile'], 'a') as outF:
            outF.write('%s %s %s\n' % (epoch, sum(lossAver) / len(lossAver), time() - start))
            outF.write('*** total %s error %s precision %s avgTime %s\n' % (count, error, precisi    on, avgTime))
        torch.save(model.state_dict(), storeName + str(epoch))
    return model


In [ ]:

criterion = nn.CrossEntropyLoss()
# optimizer_conv = optim.RMSprop(model_conv.parameters(), lr=0.01, momentum=0.9)
optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)
dst = labelFpsDataLoader(trainDirs, imgSize)
trainloader = Data.DataLoader(dst, batch_size=batchSize, shuffle=True, num_workers=8)
lrScheduler = lr_scheduler.StepLR(optimizer_conv, step_size=5, gamma=0.1)
